In [7]:
h1 = ["S" , "A"]        
h2 = ["A", "H", "S"]
h3 = ["A"]
h4 = ["F", "S"]

hashtags = [h1, h2, h3, h4]

In [14]:
def map_to_dict(hashtags):
    h_dict = {}
    for t in hashtags:
        #check length > 1
        if len(t) > 1:
            print(t)
            if len(t) == 2:
                new_t = t
            else: 
                new_t = t + [t[0]]
            print(dict(zip(new_t[:-1], new_t[1:])))
            
            
            

In [15]:
map_to_dict(hashtags)

['S', 'A']
{'S': 'A'}
['A', 'H', 'S']
{'A': 'H', 'S': 'A', 'H': 'S'}
['F', 'S']
{'F': 'S'}


### above is not great, because it doesn't work for a tweet containing four hashtags

## Using sets

In [25]:
hashtags = set(["A", "B", "C", "D"])


In [60]:

def build_edges(hashtag_set):
    #set of sets: inner sets are tuples defining an edge
    #input hashtags should be a set
    edges = set([])
    for h in hashtags:
        for remaining_h in hashtags.difference(h):
            #frozenset creates an immutable set
            edge = frozenset([h, remaining_h])
            edges.add(edge)
    return edges
            
        
            

In [61]:
build_edges(hashtags)

{frozenset({'C', 'D'}),
 frozenset({'B', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'D'}),
 frozenset({'B', 'D'}),
 frozenset({'A', 'B'})}

In [62]:
def process_tweets(hashtags):
    graph = set([])
    for hashtag_set in hashtags:
        edges = build_edges(hashtag_set)
        #add edges to graph
        for edge in edges:
            graph.add(edge)
    return graph
    
        

In [63]:
hashtags = [set(["A", "B", "C", "D"]), set(["A", "B", "E"])]

In [65]:
result = process_tweets(hashtags)
print(result)

{frozenset({'C', 'D'}), frozenset({'A', 'C'}), frozenset({'C', 'B'}), frozenset({'A', 'D'}), frozenset({'B', 'D'}), frozenset({'E', 'A'}), frozenset({'A', 'B'}), frozenset({'E', 'B'})}


In [67]:
from collections import defaultdict

h_count = defaultdict(int)
for edge in result:
    for h in edge:
        h_count[h] += 1

print(h_count)
        
        

defaultdict(<class 'int'>, {'E': 2, 'C': 3, 'D': 3, 'B': 4, 'A': 4})
